# Convolutional neural network with Tensorflow

## Introduction
TODO

## Requirements
### Imports

In [ ]:
import tensorflow as tf
import numpy as np
from deep_teaching_commons.data.fundamentals.mnist import Mnist
import matplotlib.pyplot as plt
from tqdm import tqdm

### Data

In [ ]:
train_images, train_labels, test_images, test_labels = Mnist().get_all_data(one_hot_enc=True, flatten=False)
train_images, test_images = train_images.reshape(60000, 28, 28, 1), test_images.reshape(10000,28,28,1)
print('train shapes:', train_images.shape, train_labels.shape)
print('test shapes:', test_images.shape, test_labels.shape)

### Placeholders

In [ ]:
# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
# correct answers will go here
Y = tf.placeholder(tf.float32, [None, 10])

## Convolutional Neural Network for MNIST

### Initializing the weights

In [ ]:
# convolution layer weights
W1 = tf.Variable(tf.truncated_normal([3, 3, 1, 64], stddev=0.1)) 
B1 = tf.Variable(tf.ones([64])/10)
W2 = tf.Variable(tf.truncated_normal([3, 3, 64, 32], stddev=0.1))
B2 = tf.Variable(tf.ones([32])/10)

# fully connected weights
W3 = tf.Variable(tf.truncated_normal([7 * 7 * 32, 512], stddev=0.1))
B3 = tf.Variable(tf.ones([512])/10)
W4 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
B4 = tf.Variable(tf.ones([256])/10)
W5 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
B5 = tf.Variable(tf.ones([10])/10)

### Building the Network

In [ ]:
# The model
conv1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, 2, 2, 1], padding='SAME') + B1)
conv2 = tf.nn.relu(tf.nn.conv2d(conv1, W2, strides=[1, 2, 2, 1], padding='SAME') + B2)

# reshape the output from the third convolution for the fully connected layer
flatten = tf.reshape(conv2, shape=[-1, 7 * 7 * 32])

# fully connected
dense1 = tf.nn.relu(tf.matmul(flatten, W3) + B3)
dense2 = tf.nn.relu(tf.matmul(dense1, W4) + B4)
output = tf.nn.relu(tf.matmul(dense2, W5) + B5)

In our **Use Case**, we need a kind of prediction layer on top of our output layer. We use a, so called, Softmax layer or the prediction which we put on top of the output layer. 

In [ ]:
prediction = tf.nn.softmax(output)

### Loss function and Optimizer

In general, the loss functions tells us how "good" or how "bad" our neural network is. This function is then minimized by the neural network so that the neural network gives us the best performance based on the defined loss function. For this purpose we are going to use the cross entropy loss function which is used very heavily in neural networks and seems to work very well.

**Note:** TensorFlow provides the ```softmax_cross_entropy_with_logits``` function to avoid numerical stability problems with log(0) which is NaN

In [ ]:
# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), normalised for batches of 100  images
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=Y)
cross_entropy = tf.reduce_mean(cross_entropy)*10

We are going to use the gradient descent method **Adam** to minimize our loss function. We also add a learning rate with an exponential decay. In our setting we start at a learning rate of $0.003$ and exponentially reduce it to $0.00001$.

In [ ]:
# the learning rate is: # 0.0001 + 0.003 * (1/e)^(step/2000)
learning_rate = 0.003
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

### Training the network

We define a ```accuracy``` so that we can see whether our network actually improves while training

In [ ]:
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**Hyperparameter**

In [ ]:
epochs = 10
batch_size = 256

**Main**

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
loss_history = []
train_acc_history = []
test_acc_history = []
for e in range(epochs):
    for batch_i in tqdm(range(0, train_images.shape[0], batch_size)):
        data, label = train_images[batch_i:batch_i + batch_size], train_labels[batch_i:batch_i + batch_size]
        # optimizer will not return something which is why we store it into a variable called empty
        loss, empty = sess.run([cross_entropy, train_step], feed_dict={X: data, Y: label})
        train_acc = sess.run(accuracy, feed_dict={X:train_images, Y: train_labels})
        test_acc = sess.run(accuracy, feed_dict={X:test_images, Y: test_labels})
        # append to loss history
        loss_history.append(loss)
        train_acc_history.append(train_acc)
        test_acc_history.append(test_acc)

### Evaluate model
Let us look at the optimization results. Final loss tells us how far we could reduce costs during traning process. Further we can use the first loss value as a sanity check and validate our implementation of the loss function works as intended. To visulize the whole tranings process we can plot losss values from each iteration as a loss curve. 

In [ ]:
# check loss after last and first iteration
print('last iteration loss:',loss_history[-1])
print('first iteration loss:',loss_history[0])
# Plot a loss curve
plt.plot(loss_history)
plt.ylabel('loss')
plt.xlabel('iterations')

In [ ]:
plt.plot(train_acc_history)
plt.ylabel('loss')
plt.xlabel('iterations')

In [ ]:
plt.plot(test_acc_history)
plt.ylabel('loss')
plt.xlabel('iterations')

Evaluation above gave us some inside about the optimization process but did not quantified our final model. One possibility is to calculate model accuracy.

In [ ]:
acc = sess.run(accuracy, feed_dict={X:test_images, Y: test_labels})
print(acc)

**Reset**

In [ ]:
tf.reset_default_graph()